### LLM 겍체 만들기
- ChatOpenAI 👉 OpenAI의 GPT 모델을 LangChain에서 쉽게 쓰게 해주는 래퍼(포장지)
- model_name 👉 어떤 모델을 쓸지 (gpt-4o-mini 등)
- temperature=0 👉 답변의 무작위성을 줄임(정확하고 일관된 답변)

In [29]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
question = "인프런에는 어떤 강의가 있나요?"
# llm.invoke(question)


### 간단한 프롬프트로 바로 질문하기
- 👉 llm.invoke()는
- 질문을 모델에 보내고 답변을 바로 받아오는 함수입니다.
- 👉 이건 PromptTemplate 없이도 가능합니다. (단순 호출)

In [30]:
text_prompt = f"아래의 질문에 답변해주세요:\n\n {question}"
llm.invoke(text_prompt)

AIMessage(content='인프런은 다양한 주제의 온라인 강의를 제공하는 플랫폼입니다. 주로 IT, 프로그래밍, 데이터 분석, 디자인, 마케팅, 비즈니스 등 여러 분야의 강의가 있습니다. 예를 들어, Python, Java, 웹 개발, 머신러닝, UX/UI 디자인, 디지털 마케팅 등의 강의를 찾을 수 있습니다. 강의는 초급부터 고급까지 다양한 수준으로 제공되며, 실습 중심의 강의도 많아 실제 프로젝트에 적용할 수 있는 기술을 배울 수 있습니다. 특정 강의에 대한 정보는 인프런 웹사이트를 방문하여 확인할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 134, 'prompt_tokens': 27, 'total_tokens': 161, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CTD4IjOCWecpFWpLXTqWBcnH38WqZ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--5d6e375f-fcc9-4751-b0c0-25ef1d036fc2-0', usage_metadata={'input_tokens': 27, 'output_tokens': 134, 'total_tokens': 161

### PromptTemplate (프롬프트 템플릿) 사용
- PromptTemplate 👉 프롬프트의 틀
- .invoke() 👉 {question} 자리에 실제 질문을 넣어 완성된 문자열을 반환
- llm.invoke() 👉 그 문자열을 GPT에 보냄
- 👉 결과적으로 f-string이랑 같지만, 체인화 가능하다는 게 장점입니다.

In [31]:
from langchain_core.prompts import PromptTemplate

# 1. 템플릿 정의 (한 번만 작성)
prompt_with_template = "아래의 질문에 답변해주세요:\n\n {question}"
prompt_template = PromptTemplate(template=prompt_with_template, 
                                 input_variables=["question"] )

# 2. 필요할 때마다 값만 바꿔서 사용
prompt_template.invoke({"question": question})

# 3. LLM과 결합하여 사용
llm.invoke(prompt_template.invoke({"question": question}))

AIMessage(content='인프런은 다양한 주제의 온라인 강의를 제공하는 플랫폼입니다. 주로 IT, 프로그래밍, 데이터 분석, 디자인, 마케팅, 비즈니스 등 여러 분야의 강의가 있습니다. 예를 들어, Python, Java, 웹 개발, 머신러닝, UX/UI 디자인, 디지털 마케팅 등의 강의가 포함되어 있습니다. 각 강의는 초급부터 고급까지 다양한 수준으로 제공되며, 실습 중심의 강의도 많아 실제 프로젝트에 적용할 수 있는 기술을 배울 수 있습니다. 인프런 웹사이트를 방문하면 최신 강의 목록과 상세 정보를 확인할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 135, 'prompt_tokens': 27, 'total_tokens': 162, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CTD4O0iN4e1TLyeLI36EX6y6wfZaa', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--4a0d09fd-09fe-4fc0-a9c2-f11d5e3fd795-0', usage_metadata={'input_tokens': 27, 'output_tokens': 135, 'total_tokens'

### prompt_chain.invoke({"question": "인프런에는 어떤 강의가 있나요?"})
            ↓
### PromptTemplate가 {question} 자리에 값 넣어서 문장 생성
            ↓
### 그 문장을 llm(GPT)에 자동으로 전달
            ↓
### GPT가 답변 생성
            ↓
### 결과 반환


### 체인(Chain)으로 묶기
- |(파이프) 👉 “프롬프트 → 모델 호출” 과정을 자동으로 연결
- 한 번의 .invoke()로 결과를 받을 수 있음
- 코드가 짧아지고 반복 작업에 특히 유리
- 👉 결과는 위의 llm.invoke(prompt_template.invoke(...))와 동일합니다.

In [32]:
# 체인을 만들어서 사용하면 코드가 짧아짐, invoke한번만 사용하면됨
prompt_chain = prompt_template | llm
prompt_chain.invoke({"question": question})

AIMessage(content='인프런은 다양한 주제의 온라인 강의를 제공하는 플랫폼입니다. 주로 IT, 프로그래밍, 데이터 과학, 디자인, 마케팅, 비즈니스 등 여러 분야의 강의가 있습니다. 예를 들어, Python, Java, 웹 개발, 머신러닝, UX/UI 디자인, 디지털 마케팅, 프로젝트 관리 등의 강의를 찾을 수 있습니다. 각 강의는 강사에 따라 다르며, 초급부터 고급 과정까지 다양한 수준의 강의가 마련되어 있습니다. 특정 강의에 대한 정보는 인프런 웹사이트를 방문하여 확인할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 128, 'prompt_tokens': 27, 'total_tokens': 155, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CTD4gMXnjGCViYHLGjWby7Z8WqyGn', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--df0a3e76-8e77-43c3-add8-ecfd96dfdb70-0', usage_metadata={'input_tokens': 27, 'output_tokens': 128, 'total_tokens': 155, 'input_toke

### 웹 페이지에서 문서 가져오기
- WebBaseLoader 👉 지정한 URL의 웹페이지 내용을 긁어와서 문서(Document) 객체로 변환
- loader는 웹에서 가져온 학습 자료라고 보면 됩니다.

In [13]:
%pip install -qU beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [33]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.inflearn.com/course/rag-llm-application개발-langchain/dashboard")

### 문서를 토막내기 (Chunking)
- 긴 문서를 1500자 단위로 쪼개고, 앞뒤 200자를 겹치게 저장
- 이렇게 해야 검색이 잘 됩니다 (RAG에서 핵심)
- document_list 👉 쪼개진 문서들의 리스트

In [34]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,   # 토큰수
    chunk_overlap=200, # 겹치는 토큰수
)

document_list = loader.load_and_split(text_splitter=text_splitter)

### 벡터DB에 저장 (Chroma)
- OpenAIEmbeddings 👉 문장을 숫자로 바꾸는 모델
- Chroma 👉 벡터DB (유사도 검색 가능)
- from_documents() 👉 문서를 벡터로 변환해 DB에 저장
- 나중에 retriever가 이 DB에서 관련 정보를 찾아냄
- 👉 여기서 만들어진 database가 앞에서 썼던 retriever의 기반입니다.

In [35]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

database = Chroma.from_documents(
    documents=document_list,
    embedding=embedding,
    collection_name="inflearn-lectures",
    persist_directory="./inflearn-lectures",
)


🧠 1. retriever = database.as_retriever(search_kwargs={"k": 4})
database 👉 문서를 임베딩해서 저장해둔 벡터 데이터베이스(Chroma)
.as_retriever() 👉 이 데이터베이스를 질문을 주면 관련 문서를 찾아주는 검색기로 바꿔줌
search_kwargs={"k": 4} 👉 질문과 가장 유사한 문서 4개를 가져오도록 설정
👉 즉, retriever는 이제 “질문 → 관련 문서 반환” 기능을 가진 객체가 됩니다.
👉 GPT 모델은 기본적으로 기억이 없으니, 이런 retriever가 매우 중요합니다.

🧠 2. retriever.invoke(question)
.invoke()는 “질문을 넣고 결과 받기” 메서드입니다.
question을 벡터로 변환하고, database 안의 모든 문서 벡터와 비교해서 유사도가 높은 문서를 찾아냅니다.
반환값은 문서(Document) 객체들의 리스트입니다.
👉 예를 들어 질문이 "인프런에는 어떤 강의가 있나요?"라면,
인프런 관련 페이지 내용 중에서 가장 관련성이 높은 4개 문서가 반환됩니다.

🧠 3. retrived_docs = retriever.invoke(question)
위에서 검색된 문서들을 retrived_docs라는 변수에 저장합니다.
이 변수 안에는 각 문서의 내용(page_content)과 부가정보(metadata)가 담겨 있습니다.

🧠 4. retrived_docs
검색된 문서들을 화면에 출력하는 코드입니다.
어떤 문서들이 검색됐는지 직접 확인할 수 있어요.
보통 retrived_docs[0].page_content 처럼 접근하면 첫 번째 문서의 본문만 볼 수도 있습니다.

In [36]:
retriever = database.as_retriever(search_kwargs={"k": 4})
retriever.invoke(question)
retrived_docs = retriever.invoke(question)
retrived_docs


[Document(id='3daa4826-c797-4793-8e66-08cf31320b6e', metadata={'language': 'ko', 'description': '강병진 | 실리콘밸리 GenAI 해커톤 우승자에게 배우는 RAG. 현업 노하우를 가득 담았습니다, 실리콘밸리 GenAI 해커톤 우승자가 말아주는 RAG데이터 전처리 및 효율적인 Retrieval: RAG 구성을 위해 필요한 데이터 전처리 기술과 키워드를 활용하여 검색 효율', 'source': 'https://www.inflearn.com/course/rag-llm-application개발-langchain/dashboard', 'title': 'RAG를 활용한 LLM Application 개발 (feat. LangChain) 강의 | 강병진 - 인프런'}, page_content='1) 내가 개인 llm 서비스를 만들었다고? 실화?…쉽다…\n2) 강사님은 이 많은 기술 어떻게 배운거야…내가\n계속 나아갈 수 있을까? 좌절…\n\n하지만 문득 아이디어가 떠오르더군요…\n강사님을 독촉해서 다음 강의를 만들게하자…\n기다리겠습니다! 빨리 다음 강의 부탁드려요!!\n\n실무에서 인터넷이 불가능한 환경이라 라마3로 하는 방법이 궁금합니다. 그리고 랭스미스에 대하여 잘 다루고 싶어졌습니다.host0806수강평 1 ･평균 평점 5.05.095% 수강 후 작성간단한 챗봇을 만들어보려고 며칠을 챗지피티와 씨름했던 시간이 무색하게 정말 이해하기 쉽게 개발 과정을 배울 수 있었습니다.\n혼자 공부할 때와는 다르게 어떤게 어디에 필요한지 알 수 있어 너무 좋았습니다. 윈도우 사용자도 크게 어렵지 않게 따라갈 수 있었습니다. 추천합니다JAY probio수강평 7 ･평균 평점 5.05.069% 수강 후 작성짧고 강력한 강의입니다.\n핵심만 쏙쏙 뽑아서 전달하는 강의입니다.\n다른 강의 들어보니까 돌려돌려 설명해서 더 어렵기만하고 이해도 안되고 ㅠㅠㅠㅠㅠㅠ 다른 강의도 많이 만들어주세요Dominus Mr.수강평 6

### RAG용 PromptTemplate 만들기
- context에는 retriever로 가져온 문서 내용이 들어감
- question에는 사용자의 질문이 들어감
- 모델에게 “문맥 기반으로 답해라”라고 지시
- 👉 이것이 일반 프롬프트와 다른 점: 단순 질문이 아니라 **관련 문서(context)**와 함께 보냄

In [39]:
rag_prompt_template = PromptTemplate(
    template='''
    you are a helpful assistant that can answer questions about the Inflearn website.
    You are given the following context:
    {context}
    Question: {question}
    ''',
    input_variables=["context", "question"]
)

rag_chain = rag_prompt_template | llm
rag_chain.invoke({"context": retrived_docs, "question": question})


AIMessage(content='인프런에는 "RAG를 활용한 LLM Application 개발 (feat. LangChain)"이라는 강의가 있습니다. 이 강의는 강병진 강사가 진행하며, 실리콘밸리 GenAI 해커톤 우승자의 경험을 바탕으로 RAG 데이터 전처리 및 효율적인 Retrieval에 대한 내용을 다룹니다. 강의에서는 개인 LLM 서비스를 만드는 방법, 데이터 전처리 기술, 검색 효율을 높이는 키워드 활용 등에 대한 실무 노하우를 배울 수 있습니다.\n\n강의 링크는 [여기](https://www.inflearn.com/course/rag-llm-application개발-langchain/dashboard)에서 확인할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 148, 'prompt_tokens': 2563, 'total_tokens': 2711, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 2432}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CTD6yBkfX9FuAmjptSRDyfcYAZaZw', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--b64fec2b-9b13-4b6d-aad4-9e567402deb4-0', usa

ImportError: cannot import name 'hub' from 'langchain' (/Users/kimdonghyeon/.pyenv/versions/rng-venv/lib/python3.10/site-packages/langchain/__init__.py)